# 🖼️ Smart Gallery Backend

Production-ready FastAPI backend with:
- **YOLO v8** - Object detection
- **InsightFace** - Face detection & recognition
- **CLIP** - Semantic search

Run all cells to start the server with ngrok tunnel.

In [ ]:
# Mount Google Drive (optional - for persistent storage)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install dependencies
!pip install -q fastapi==0.109.2 uvicorn[standard]==0.27.1 python-multipart==0.0.9
!pip install -q pydantic==2.6.1 pydantic-settings==2.1.0
!pip install -q aiosqlite==0.19.0 sqlalchemy==2.0.25
!pip install -q Pillow==10.2.0 opencv-python-headless==4.9.0.80
!pip install -q ultralytics==8.1.0
!pip install -q insightface==0.7.3 onnxruntime-gpu==1.17.0
!pip install -q open-clip-torch==2.24.0
!pip install -q faiss-cpu==1.7.4
!pip install -q aiofiles==23.2.1 pyngrok==7.0.0
print("✅ Dependencies installed!")

In [ ]:
# Clone or upload the backend code
# Option 1: Upload the smart-gallery-backend folder to /content/
# Option 2: Clone from GitHub (replace with your repo)
# !git clone https://github.com/yourusername/smart-gallery-backend.git /content/smart-gallery-backend

import os
os.chdir('/content')

# Check if backend exists
if os.path.exists('/content/smart-gallery-backend'):
    print("✅ Backend code found!")
else:
    print("❌ Please upload smart-gallery-backend folder to /content/")

In [ ]:
# Configuration
import os

# Set environment variables
os.environ['DEBUG'] = 'false'
os.environ['DEVICE'] = 'cuda'  # or 'cpu'

# Use Google Drive for persistent storage (optional)
# os.environ['DATA_DIR'] = '/content/drive/MyDrive/smart_gallery_data'

# Custom YOLO model path (optional)
# os.environ['YOLO_MODEL_PATH'] = '/content/drive/MyDrive/models/yolov8x_custom.pt'

print("✅ Configuration set!")

In [ ]:
# Setup ngrok for public access
from pyngrok import ngrok, conf
import getpass

# Get ngrok auth token
print("Get your ngrok auth token from: https://dashboard.ngrok.com/get-started/your-authtoken")
NGROK_TOKEN = getpass.getpass("Enter ngrok auth token: ")

# Configure ngrok
conf.get_default().auth_token = NGROK_TOKEN
print("✅ ngrok configured!")

In [ ]:
# Start the server
import sys
import asyncio
import threading
import time

sys.path.insert(0, '/content/smart-gallery-backend')

def run_server():
    import uvicorn
    uvicorn.run(
        "app.main:app",
        host="0.0.0.0",
        port=8000,
        log_level="info"
    )

# Start server in background thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Wait for server to start
print("⏳ Waiting for server to initialize (loading ML models)...")
time.sleep(30)  # Give time for models to load

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print("\n" + "="*60)
print("🚀 SMART GALLERY API IS RUNNING!")
print("="*60)
print(f"\n🌐 Public URL: {public_url}")
print(f"📚 API Docs: {public_url}/docs")
print(f"\n📤 Upload photos via: POST {public_url}/photos/upload")
print(f"🔍 Search photos: GET {public_url}/search/text?q=your+query")
print("\n" + "="*60)

In [ ]:
# Test the API
import requests

# Get the ngrok URL
url = str(public_url).replace('NgrokTunnel: "', '').split('"')[0]

# Test health endpoint
response = requests.get(f"{url}/health")
print("Health check:", response.json())

# Test stats endpoint
response = requests.get(f"{url}/stats")
print("\nStats:", response.json())

In [ ]:
# Upload a test image
import requests
from google.colab import files

# Upload image from your computer
print("Select an image to upload:")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    file_content = uploaded[filename]
    
    # Upload to API
    url = str(public_url).replace('NgrokTunnel: "', '').split('"')[0]
    response = requests.post(
        f"{url}/photos/upload",
        files={"files": (filename, file_content)}
    )
    
    result = response.json()
    print("\n✅ Upload result:")
    print(f"Success: {result['success']}")
    
    if result['photos']:
        photo = result['photos'][0]
        print(f"\nPhoto ID: {photo['id']}")
        print(f"Detections: {len(photo['detections'])} objects")
        print(f"Faces: {len(photo['faces'])} faces")
        
        if photo['detections']:
            print("\nDetected objects:")
            for det in photo['detections']:
                print(f"  - {det['class_name']}: {det['confidence']:.2%}")
        
        if photo['faces']:
            print("\nDetected faces:")
            for face in photo['faces']:
                age = face.get('age', 'unknown')
                gender = face.get('gender', 'unknown')
                print(f"  - Age: {age}, Gender: {gender}")

In [ ]:
# Test semantic search
import requests

url = str(public_url).replace('NgrokTunnel: "', '').split('"')[0]

# Search by text
query = "person smiling"  # Change this to your search query
response = requests.get(f"{url}/search/text", params={"q": query})

results = response.json()
print(f"Search results for '{query}':")
print(f"Found {results['total']} matches")

for r in results['results'][:5]:
    print(f"  - {r['photo']['original_filename']}: {r['similarity']:.2%}")

In [ ]:
# Keep the notebook running
# The server will stay active as long as this cell is running
import time

print("\n🟢 Server is running...")
print(f"📍 URL: {public_url}")
print("\nPress the stop button to shutdown the server.")

try:
    while True:
        time.sleep(60)
        print(".", end="", flush=True)
except KeyboardInterrupt:
    print("\n\n🛑 Server stopped.")